In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub
import sentencepiece as spm
import numpy as np
from absl import logging

# Load Model From TF Hub

In [6]:
module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-lite/2")

In [7]:
input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
encodings = module(
    inputs=dict(
        values=input_placeholder.values,
        indices=input_placeholder.indices,
        dense_shape=input_placeholder.dense_shape))


with tf.Session() as sess:
    spm_path = sess.run(module(signature="spm_path"))

sp = spm.SentencePieceProcessor()
with tf.io.gfile.GFile(spm_path, mode="rb") as f:
    sp.LoadFromSerializedProto(f.read())
# it looks liek that model is already in the downloaded file
print("SentencePiece model loaded at {}.".format(spm_path))


def process_to_IDs_in_sparse_format(sp, sentences):
    # An utility method that processes sentences with the sentence piece processor
    # 'sp' and returns the results in tf.SparseTensor-similar format:
    # (values, indices, dense_shape)
    ids = [sp.EncodeAsIds(x) for x in sentences]
    max_len = max(len(x) for x in ids)
    dense_shape=(len(ids), max_len)
    values=[item for sublist in ids for item in sublist]
    indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
    return (values, indices, dense_shape)


# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
messages = [word]

values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, messages)

# Reduce logging output.
#logging.set_verbosity(logging.ERROR)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(
      encodings,
      feed_dict={input_placeholder.values: values,
                input_placeholder.indices: indices,
                input_placeholder.dense_shape: dense_shape})

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        print(message_embedding)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


SentencePiece model loaded at b'C:\\Users\\Preston\\AppData\\Local\\Temp\\tfhub_modules\\539544f0a997d91c327c23285ea00c37588d92cc\\assets\\universal_encoder_8k_spm.model'.
Message: Elephant
Embedding size: 512
[0.053387485444545746, 0.05319438874721527, -0.05235603451728821, -0.047084346413612366, -0.053246673196554184, -0.004174722824245691, 0.044532790780067444, -0.05203130468726158, 0.023275665938854218, 0.04738346114754677, 0.009968076832592487, 0.021561693400144577, -0.052824605256319046, -0.05354135483503342, -0.04973248392343521, 0.050568837672472, 0.025157103314995766, 0.053211648017168045, 0.05357431247830391, -0.04986065626144409, 0.037697676569223404, 0.05177415907382965, 0.008676332421600819, -0.04257824271917343, -0.025872722268104553, 0.05238695442676544, 0.030760573223233223, 0.05175681784749031, 0.00743406405672431, 0.05349412187933922, 0.007445307448506355, -0.05353155359625816, -0.03697839751839638, -0.05346200242638588, 0.03463117033243179, 0.025551211088895798, -0.0

# Load Model Locally After Downloading

In [8]:
import tarfile

# Open downloaded TensorFlow Hub Model
file = tarfile.open('universal-sentence-encoder-lite_2.tar.gz')
  
# extracting file
file.extractall('./USE_lite_2')
file.close()




import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub
import sentencepiece as spm

# load the extracted model
module = hub.Module('USE_lite_2')


# initialize model with placeholders
input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
encodings = module(
    inputs=dict(
        values=input_placeholder.values,
        indices=input_placeholder.indices,
        dense_shape=input_placeholder.dense_shape))



# Load sentence piece model locally
# sentence piece model is needed for tokenization
with tf.Session() as sess:
    spm_path = sess.run(module(signature="spm_path"))

sp = spm.SentencePieceProcessor()
with tf.io.gfile.GFile(spm_path, mode="rb") as f:
    sp.LoadFromSerializedProto(f.read())
# it looks liek that model is already in the downloaded file
print("SentencePiece model loaded at {}.".format(spm_path))



# Compute the embedding for a word or sentnce
word = "Test"
messages = [word]

values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, messages)

# Reduce logging output.
logging.set_verbosity(logging.ERROR)
embed_list = []
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(
      encodings,
      feed_dict={input_placeholder.values: values,
                input_placeholder.indices: indices,
                input_placeholder.dense_shape: dense_shape})

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        print(message_embedding)
        embed_list = message_embedding

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


SentencePiece model loaded at b'USE_lite_2\\assets\\universal_encoder_8k_spm.model'.
Message: Test
Embedding size: 512
[0.038433972746133804, -0.04892175272107124, -0.04413247108459473, -0.04872800409793854, -0.04622480645775795, -0.022745328024029732, 0.048912107944488525, -0.048922739923000336, 0.044430870562791824, 0.034504521638154984, 0.048912957310676575, 0.04804450273513794, -0.03301442414522171, 0.04585035517811775, -0.04812264069914818, 0.048761431127786636, -0.04720494896173477, -0.04494010657072067, -0.048926327377557755, 0.048916708678007126, -0.04863056540489197, 0.04892471432685852, -0.0053505608811974525, 0.038418956100940704, 0.04863739386200905, -0.04871702566742897, -0.04618111625313759, 0.04739350453019142, -0.04892611876130104, 0.048407766968011856, -0.048484060913324356, -0.04891733080148697, -0.045919645577669144, -0.04892629384994507, 0.048012129962444305, -0.04250679165124893, -0.04892531409859657, 0.04878542199730873, -0.034613776952028275, 0.037462230771780014

### Zip the model

In [9]:
import os
import zipfile

# create the zipped version
zf = zipfile.ZipFile("USE_lite_2.zip", "w")
for dirname, subdirs, files in os.walk("USE_lite_2"):
    zf.write(dirname)
    for filename in files:
        zf.write(os.path.join(dirname, filename))
zf.close()
